<a href="https://colab.research.google.com/github/ikonthomas/SchoolPlay/blob/master/MANGO_News_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Libraries Importing
import re
import nltk
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor,LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')


Using TensorFlow backend.


In [0]:
#Data Importing Returns_News file
#we'll use all of the dates up to the end of 2014 as our training data and 2015 going forward as testing data.

data = pd.read_csv('https://raw.githubusercontent.com/ikonthomas/SchoolPlay/master/Returns_News.csv')
#train = data[data['Date'] < '2015/01/01']
#test = data[data['Date'] > '2014/12/31']
train = pd.read_csv('https://raw.githubusercontent.com/ikonthomas/SchoolPlay/master/Returns_News_Train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/ikonthomas/SchoolPlay/master/Returns_News_Test.csv')

In [0]:
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10
0,02/01/2009,1,b'Australia refuses Bush administration reques...,b'Propaganda war: trusting what we see?',"b""CNN: US weapons 'killing innocent civilians'...","b'""Let us not forget that it was Israel, which...","b""Northern Ireland man discovers 'secret of in...",b'Lynndie England: Life after Abu Ghraib',b'Zimbabwean lawyer: Law has broken down - CNN...,b'Nobel Peace Prize laureate Shirin Ebadi of I...,"b'Beijing moves to stifle reform calls, called...","b'Israels Goal - Ending Hamas Rule, Rocket Fir..."
1,05/01/2009,1,b'Palestine Girl stands up to Israel Soldier ',b'Israeli Strike Kills 40 at U.N. run School i...,"b'Norwegian doctor in Gaza: ""This is an all-ou...","b'Between 600,000 and 700,000 Gazans have no w...","b'Ron Paul: ""No one mentions the tragedy in Ga...",b'UK police have been given the power to hack ...,"b""Strike on Gaza school 'kills 40'""","b""GAZA PIC's. WARNING :VERY GRAPHIC!!!!!!!!!""","b'""We are wading in death, blood, and amputees...",b'Several European countries say they have suf...
2,06/01/2009,1,b'Anyone know if this is true?',"b""British Gov't wants private firms to build $...",b'Israeli Tank fire kills dozens at UN School....,"b'Bush Aides: ""Historians will judge Bush."" Re...","b""Russia has cut off all Europe's gas supplies...","b'""Israel, not the Palestinians, has opposed i...",b'Venezuela expels Israel envoy over Gaza atta...,b'This is what happens on the streets of Israe...,b'Israeli soldiers entered their building and ...,"b'""Almost a third of the 689 Palestinians kill..."
3,07/01/2009,0,b'You are being lied to about pirates',b'Rockets hit Israel from Lebanon',b'I arrived in Israel 40 years ago. It took me...,"b'Red Cross finds four small children in Gaza,...",b'UN: Israel kills driver on aid mission to Gaza',b'Israel kills driver of UN truck on aid missi...,"b'These photos, taken in Gaza in 2006, show th...","b""The Bush Administration's Most Despicable Act""","b""TED: Hans Rosling: Debunking third-world myt...",b'Gaza is full-blown humanitarian crisis'
4,08/01/2009,1,b'Vatican: Gaza is a giant concentration camp',b'Israel shelled Gaza house crowded with civil...,b'Red Cross: Israel breaking international law...,b'Israeli army ordered 70 persons of extended ...,b'Israel Faces Mounting Outcry on Gaza War - N...,"b'Israel bombs a shelter, kills 30 civilians'",b'Israel accused of delaying medical access to...,b'Israel to continue offensive despite UN reso...,"b""The Army's Newest Recruitment Technique is a...","b""Atheist Bus Campaign is 'offensive', say 50 ..."


In [0]:
train.tail(10)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10
1500,17/12/2014,1,The United States will normalize diplomatic re...,Iraq: 150 women executed after refusing to mar...,Foreign exchange in Finland no longer recogniz...,Russia says Sweden smokes too much Weed and th...,"About 15,000 people show up for Anti-Islam mar...",Russians hurried to change their savings and p...,Iran had warned that Sydney attacker was unsta...,Pakistan Lifts Death Penalty Ban After Massacre,Two Blasts Outside Girls College Near Peshawar,Putin's cronies lose $50 billion
1501,18/12/2014,1,Iran this week unveiled a monument to Jewish s...,Activists to Drop 'Interview' DVDs Over North ...,"Curiosity Rover Drills Into Mars Rock, Finds W...",Vladimir Putin compares Russia to cornered bea...,"Qatar hires ""$8 fake fans"" to fill empty stadi...",Pope Francis played key role in U.S.-Cuba deal,Belarussian President Demands Russia Trade Be ...,Navajos reclaim sacred masks at Paris auction,Nearly 97 percent of the airstrikes against IS...,"Zakiur Rehman Lakhvi, the main accused in the ..."
1502,19/12/2014,1,Claims that boys were murdered by VIP sex ring...,Kurds recapture large area from ISIS,Sydney's chief Islamic funeral director yester...,Apple is failing to protect workers in Chinese...,A suitcase-sized Canadian satellite whose fund...,Ukraine forms 'ministry of truth' to regulate ...,Archaeologists working on a site near Stonehen...,Seabirds 'steer clear' of offshore wind farms....,"General Motor, Audi and Jaguar halt car sales ...",80% of British Public Want Greens Included in ...
1503,22/12/2014,1,"Pirate Bay Domain Back Online, Waving a Pirate...","Argentine Court rules: Orang Utans are ""non-hu...",Peru Is Now Giving Free Solar Power To Its 2 M...,"New York Times calls for Cheney, Bush official...",Pope Francis issued a blistering critique Mond...,Polio has Almost Been Completely Eradicated in...,CIA's how to be a travelling spy guide publish...,"NKorea threatens strikes on US, warned that it...",North Korea Now Claims the US Government Made ...,UK officials named in pedophile dossier
1504,23/12/2014,1,North Korea's internet is offline; massive DDO...,SAUDI OIL MINISTER: I Don't Care If Prices Cra...,ISIL sex slavery has pushed women and girls fr...,Ukraine votes to drop neutral NATO status,The United States on Monday slammed North Kore...,Oil spill sullies World Heritage Site home of ...,"Russian diplomat: ""Ukraine's renunciation of i...","$300,000 in gold missing from Ukraine Central ...",Tunisia election: anti-Islamist party claims v...,US Corporations claim billion in Assets in Cub...
1505,24/12/2014,1,Death toll among Qatars 2022 World Cup workers...,Fishing Supertrawlers to be banned permanently...,Indian telecommunications company Airtel viola...,North Korea's Internet is down again; second b...,Jakarta to ban virginity tests for female civi...,Workers at Chinese iPhone factories are pressu...,Thousands in Spain protest ban on demonstratio...,London activists occupy 5-storey former bank i...,"Russian Inflation Passes 10%, Indicates Pricin...",Australia records biggest emissions drop in a ...
1506,26/12/2014,1,Saudis are eagerly awaiting the approval of a ...,"Due to the fall in oil prices, Saudi Arabia is...",Bill giving government the power to shutdown t...,A struggle for women's rights is brewing withi...,Putin cancels New Year's Holiday for governmen...,Ireland is siding with Microsoft in its legal ...,More than 80 people including women and childr...,"The Syrian Army has killed nearly 5,000 Islami...",ISIS governor of Mosul killed in airstrikes,Court clears way for clinically dead pregnant ...
1507,29/12/2014,0,Solar Power Storage Prices Drop 25% In Germany...,North Korea Hit Again By Internet Outage; Expe...,ARCHAEOLOGY - Massive ancient underground city...,Reopen investigation into Westminster pedophil...,"Taliban declare 'defeat' of U.S., allies in Af...",British Falklands defence review after militar...,Virgin Confirms Fault On 

In [0]:
test.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10
0,02/01/2015,1,Most cases of cancer are the result of sheer b...,Iran dismissed United States efforts to fight ...,Poll: One in 8 Germans would join anti-Muslim ...,UK royal family's Prince Andrew named in US la...,Some 40 asylum-seekers refused to leave the bu...,Pakistani boat blows self up after India navy ...,Sweden hit by third mosque arson attack in a week,940 cars set alight during French New Year,Salaries for top CEOs rose twice as fast as av...,"Norway violated equal-pay law, judge says: Jud..."
1,05/01/2015,0,Moscow-&gt;Beijing high speed train will reduc...,Two ancient tombs were discovered in Egypt on ...,China complains to Pyongyang after N Korean so...,Scotland Headed Towards Being Fossil Fuel-Free...,Prime Minister Shinzo Abe said Monday he will ...,Sex slave at centre of Prince Andrew scandal f...,Gay relative of Hamas founder faces deportatio...,The number of female drug addicts in Iran has ...,"After Decades of Searching, the Causeway for t...",India lost 64 tigers in 2014
2,06/01/2015,0,US oil falls below $50 a barrel,"Toyota gives away 5,680 fuel cell patents to b...",Young Indian couple who had been granted polic...,A senior figure in Islamic States self-declare...,Fukushima rice passes radiation tests for 1st ...,Nearly all Spanish parties guilty of financial...,King Abdullah to abdicate Saudi Throne,Taliban Commander Caught Networking On LinkedIn,Mexican missing students: mayor's wife charged...,New York Times reporter James Risen refused on...
3,07/01/2015,1,'Shots fired' at French magazine HQ,90% of Bibi Netanyahus Campaign Funds From U.S.,ISIS behead street magician for entertaining c...,Denmark sets new world record in renewable ene...,"ISIS Closes Schools In Syria, Leaving 670000 C...",The European Union wants Turkey to explain how...,10 people have died in a shooting at the headq...,Most Earth-like planet ever discovered could b...,New questions raised about U.S.-Saudi relation...,Massive hunt underway for gunmen in Paris terr...
4,08/01/2015,1,New Charlie Hebdo issue to come out next week:...,"Ahmed Merabet, Cop Killed In Paris Attacks, Wa...",Youngest Suspect in Charlie Hebdo Attack Turns...,"Grenades thrown at a mosque in Le Mans, west o...",Danish newspaper to print Charlie Hebdo cartoo...,Muslim politician from India offers $8M to Cha...,Vladimir Putin has condemned a deadly terroris...,Turkey is trying to hide any evidence that hol...,Danish Paper That Published 12 Muhammad Cartoo...,Two Charlie Hebdo attackers located in north F...


In [0]:
#Processing our Data
#Transforming the news headlines into the number of words.

trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))
basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)


(1510, 20918)


In [0]:

#Algorithm: Logic Regression

#Input: the counts of single words

basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(basictrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
basictest = basicvectorizer.transform(testheadlines)
preds1 = basicmodel.predict(basictest)
acc1=accuracy_score(test['Label'], preds1)
print('Logistic Regression 1 accuracy: ',acc1 )

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression 1 accuracy:  0.4880952380952381


The accuracy is only 49%

In [0]:
basicwords = basicvectorizer.get_feature_names()
basiccoeffs = basicmodel.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : basicwords, 
                        'Coefficient' : basiccoeffs})
coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
coeffdf.head()

,Word,Coefficient
17375,so,0.777398
11422,makes,0.746689
1512,another,0.605534
8632,hamas,0.573705
20339,we,0.554730


In [0]:
coeffdf.tail()

,Word,Coefficient
17996,strike,-0.552088
9742,injured,-0.582503
3812,chief,-0.599710
19343,turkey,-0.665417
4767,country,-0.718426


Logistic Regression 2


Input: the counts of phrases with two connected words(exclude words which are too common like "a" ,"an" ,"the" and words too uncommon of which counts are too small )

We delete phrases of which frequency lower than 0.03 or higher than 0.97



In [0]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.97, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1510, 178)


In [0]:
advancedmodel = LogisticRegression()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds2 = advancedmodel.predict(advancedtest)
acc2=accuracy_score(test['Label'], preds2)
print('Logistic Regression 2 accuracy: ', acc2)

Logistic Regression 2 accuracy:  0.5357142857142857


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


We see an improvement when using phrases that previously with single words


In [0]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head()

,Words,Coefficient
110,than the,1.641290
101,right to,1.415258
85,of an,1.324453
10,and other,1.252696
168,we are,1.124815


In [0]:
advcoeffdf.tail()

,Words,Coefficient
174,world war,-0.908041
119,the last,-0.918783
145,to give,-0.947991
43,he was,-0.983150
123,the police,-1.012975


**Logic Regression 3**

Input: the counts of phrases with three connected words(exclude words which are too common like "a" ,"an" ,"the" and words too uncommon of which counts are too small )

In [0]:
advancedvectorizer = TfidfVectorizer( min_df=0.0039, max_df=0.1, max_features = 200000, ngram_range = (3, 3))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1510, 770)


In [0]:
advancedmodel = LogisticRegression()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds3 = advancedmodel.predict(advancedtest)
acc3 = accuracy_score(test['Label'], preds3)
print('Logic Regression 3 accuracy: ', acc3)

Logic Regression 3 accuracy:  0.5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


The accuracy is slightly lower at 50% than input of phrases with two connected words.

In [0]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head()

,Words,Coefficient
723,up in arms,1.068703
191,he has been,1.051623
301,it turns out,0.988381
568,the lives of,0.983618
142,for all the,0.979222


**Naive Bayes**

NB 1

In [0]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.7, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1510, 180)


In [0]:
advancedmodel = MultinomialNB(alpha=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds4 = advancedmodel.predict(advancedtest)
acc4=accuracy_score(test['Label'], preds4)
print('NBayes 1 accuracy: ', acc4)

NBayes 1 accuracy:  0.5317460317460317


In [0]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head()

,Words,Coefficient
142,that,-4.248585
59,has,-4.312148
13,as,-4.324112
27,by,-4.348073
51,from,-4.389266


In [0]:
advcoeffdf.tail()

,Words,Coefficient
139,stop,-5.805594
43,european,-5.812780
44,even,-5.813698
172,without,-5.848278
19,bbc,-5.893120


Naive Bayes 2


In [0]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1510, 171)


In [0]:
advancedmodel = MultinomialNB(alpha=0.0001)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds5 = advancedmodel.predict(advancedtest)
acc5 = accuracy_score(test['Label'], preds5)
print('NBayes 2 accuracy: ', acc5)

NBayes 2 accuracy:  0.5079365079365079


In [0]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head()

,Words,Coefficient
79,north korea,-4.132569
39,has been,-4.149692
169,year old,-4.223944
40,have been,-4.327065
36,from the,-4.328559


In [0]:
advcoeffdf.tail()

,Words,Coefficient
32,end of,-5.809781
139,to give,-5.817873
114,the israeli,-5.832433
119,the police,-5.885780
167,world war,-5.955652


## Random Forest

In [0]:
#RF 1
advancedvectorizer = TfidfVectorizer( min_df=0.01, max_df=0.99, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1510, 2460)


In [0]:
advancedmodel = RandomForestClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds6 = advancedmodel.predict(advancedtest)
acc6 = accuracy_score(test['Label'], preds6)
print('RF 1 accuracy: ', acc6)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF 1 accuracy:  0.4642857142857143


RandomForest 2

In [0]:
#RF 2
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1510, 171)


In [0]:
advancedmodel = RandomForestClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds7 = advancedmodel.predict(advancedtest)
acc7 = accuracy_score(test['Label'], preds7)
print('RF 2 accuracy: ', acc7)

RF 2 accuracy:  0.5


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


## Gradient Boosting Machines
GBM 1


In [0]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.9, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1510, 183)


In [0]:
advancedmodel = GradientBoostingClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds8 = advancedmodel.predict(advancedtest.toarray())
acc8 = accuracy_score(test['Label'], preds8)
print('GBM 1 accuracy: ', acc8)

GBM 1 accuracy:  0.5833333333333334
